In [1]:
import os
import sys

In [2]:
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname('__file__'), '..')))
from src.encoding.encoders import *
from src.encoding.tools import create_encoding_pipeline
import pandas as pd
import src.features as ft
from src.datasets.base_tabular_dataset import BaseTabularDataset
import datetime as dt
import logging
import pathlib
pd.set_option('display.max_columns', None)

In [3]:
# Define a logger used by all modules
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO, encoding="utf-8",
                    format="%(name)s %(asctime)s: %(levelname)s: %(message)s", handlers=[logging.StreamHandler()])

In [4]:
# Define the root directory of the project
root_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
root_dir = pathlib.Path(root_dir)

In [5]:
# Define the configuration for the fetching of the data
fetch_config = {
    "data_start": '01-01-2017',
    "data_stop": '31-12-2023',
    'data_dir': root_dir / 'data',
    "etablissement": "CHU Dijon",
    "departement": "21",
    'region': 'BOURGOGNE'
    }

In [6]:
# Select the features to be used in the dataset
ars_features_class = [
    ft.HopitalFeatures,
    ft.AirQualityFeatures,
    ft.EpidemiologicalFeatures,
    # ft.FireFightersFeatures(include_calls=False),
    ft.GoogleTrendFeatures,
    ft.MeteorologicalFeatures,
    ft.SociologicalFeatures,
    ft.SportsCompetitionFeatures,
    ft.TrafficFeatures
    ]

In [7]:
# Select the target columns to be predicted
# target_colomns = ['nb_vers_hospit']
target_colomns = ['Total_CHU Dijon']

In [8]:
# Define an encoding scheme to create the encoding pipeline
encoders_dict = {
    'number': {
        'as_number': {
            'imputers': [imputers.SimpleImputer(strategy='mean')],
            'encoders': [
                ne.StandardScaler(),
            ]
        }
    },
    'category': {
        'as_category': {
            'imputers': [imputers.SimpleImputer(strategy='most_frequent')],
            'encoders': [
                ne.MultiTargetEncoder(drop_invariant=True, return_df=True),
            ]
        }
    },
    'datetime': {
        'as_number': {
            'imputers': [de.DateFeatureExtractor()],
            'encoders': [
                ne.CyclicalFeatures(drop_original=True)
            ]
        },
        'as_category': {
            'imputers': [de.DateFeatureExtractor(dtype='category')],
            'encoders': [
                ne.MultiTargetEncoder(drop_invariant=True, return_df=True),
            ]
        }
    }
}

In [9]:
# Create the encoding pipeline
pipeline = create_encoding_pipeline(encoders_dict)

Creating encoding pipeline


In [10]:
# Define the splitting scheme to create the sets
split_config = {'test_size': 0.2, 'val_size': 0.2, 'shuffle': False}

In [11]:
# Define the configuration of the dataset
dataset_config = {
    'from_date': '15-01-2019',
    'to_date': '31-12-2023',
    'shift': range(1, 14, 1),
    'rolling_window': [7, 14],
    'freq': '1D',
    'split_config': split_config,
    'create_X_y': True,
    'encoding_pipeline': pipeline,
    'targets_names': target_colomns,
    'targets_shift': -7,
    'targets_rolling_window': 7,
    'targets_history_shifts': range(1, 14, 1),
    'targets_history_rolling_windows': [7, 14],
    'drop_constant_thr':1.0,
    'data_dir': root_dir / 'data'
    }

In [12]:
baseTabularDataset = BaseTabularDataset(features_class=ars_features_class, logger=logger, fetch_config=fetch_config, getter_config=dataset_config)
# print(baseTabularDataset.data.columns.to_list())
# liste = ['O3_FR26005%%mean_7J', 'target_Total_CHU Dijon']
# baseTabularDataset = baseTabularDataset.get_dataset(**dataset_config, features_names=liste)
# baseTabularDataset.train_set.to_csv('train.csv', index=False)
# baseTabularDataset.test_set.to_csv('test.csv', index=False)
# baseTabularDataset.val_set.to_csv('val.csv', index=False)

root 2024-10-15 17:12:20,541: INFO: Initialisation de la classe BaseTabularDataset
root 2024-10-15 17:12:20,544: INFO: Initialisation de la classe hopitalfeatures
root 2024-10-15 17:12:20,545: INFO: Initialisation de la classe airqualityfeatures
root 2024-10-15 17:12:20,546: INFO: Initialisation de la classe epidemiologicalfeatures
root 2024-10-15 17:12:20,546: INFO: Initialisation de la classe googletrendfeatures
root 2024-10-15 17:12:20,546: INFO: Initialisation de la classe meteorologicalfeatures
root 2024-10-15 17:12:20,547: INFO: Initialisation de la classe sociologicalfeatures
root 2024-10-15 17:12:20,547: INFO: Initialisation de la classe sportscompetitionfeatures
root 2024-10-15 17:12:20,548: INFO: Initialisation de la classe trafficfeatures
root 2024-10-15 17:12:20,549: INFO: Fetching data for hopitalfeatures


/home/maxime/Documents/WORKSPACES/forecasting_models/data


root 2024-10-15 17:12:20,801: INFO: Fetching data for airqualityfeatures
root 2024-10-15 17:12:20,812: INFO: Fetching data for epidemiologicalfeatures
root 2024-10-15 17:12:24,551: INFO: Fetching data for googletrendfeatures
root 2024-10-15 17:12:24,562: INFO: Fetching data for meteorologicalfeatures


/home/maxime/Documents/WORKSPACES/forecasting_models/tests
/home/maxime/Documents/WORKSPACES/forecasting_models/src/geolocalisation


The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


root 2024-10-15 17:12:25,273: INFO: Fetching data for sociologicalfeatures
root 2024-10-15 17:12:26,321: INFO: Fetching data for sportscompetitionfeatures
root 2024-10-15 17:12:26,324: INFO: Fetching data for trafficfeatures
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


root 2024-1

Dropped constant columns from both sets: {'NO2_FR26094%%J-2', 'PM10_FR26094%%J-5', 'PM25_FR26094%%std_14J', 'meteo_CHU DIJON_0_snow%%J-4', 'meteo_CHU DIJON_0_snow%%J-7', 'NO2_FR26094%%mean_7J', 'meteo_CHU DIJON_0_snow%%J-13', 'meteo_CHU DIJON_0_snow%%J-9', 'PM10_FR26010%%J-7', 'PM10_FR26010%%J-3', 'NO2_FR26094%%J-1', 'meteo_CHU DIJON_0_snow%%J-2', 'NO2_FR26094%%J-5', 'NO2_FR26094%%J-11', 'PM10_FR26010%%J-5', 'PM10_FR26010%%std_7J', 'PM10_FR26010%%J-11', 'NO2_FR26010%%std_7J', 'PM25_FR26094%%std_7J', 'NO2_FR26094%%std_14J', 'PM10_FR26010%%J-1', 'PM10_FR26094%%std_14J', 'PM10_FR26094%%J-10', 'PM10_FR26010%%J-6', 'PM10_FR26094%%J-9', 'PM10_FR26094%%J-11', 'NO2_FR26094', 'PM10_FR26010%%J-2', 'PM10_FR26094%%J-6', 'NO2_FR26094%%J-13', 'PM10_FR26010', 'meteo_CHU DIJON_0_snow%%J-3', 'meteo_CHU DIJON_0_snow%%J-8', 'NO2_FR26010%%std_14J', 'NO2_FR26094%%J-3', 'PM10_FR26094', 'NO2_FR26094%%J-10', 'PM10_FR26010%%mean_14J', 'meteo_CHU DIJON_0_snow', 'NO2_FR26094%%J-4', 'meteo_CHU DIJON_0_snow%%std_1

In [13]:
data = baseTabularDataset.data
data

,Total_CHU Dijon,HNFC_moving,nb_vers_hospit,Total_CHU Dijon%%J-1,Total_CHU Dijon%%J-2,Total_CHU Dijon%%J-3,Total_CHU Dijon%%J-4,Total_CHU Dijon%%J-5,Total_CHU Dijon%%J-6,Total_CHU Dijon%%J-7,Total_CHU Dijon%%J-8,Total_CHU Dijon%%J-9,Total_CHU Dijon%%J-10,Total_CHU Dijon%%J-11,Total_CHU Dijon%%J-12,Total_CHU Dijon%%J-13,Total_CHU Dijon%%mean_7J,Total_CHU Dijon%%std_7J,Total_CHU Dijon%%mean_14J,Total_CHU Dijon%%std_14J,HNFC_moving%%J-1,HNFC_moving%%J-2,HNFC_moving%%J-3,HNFC_moving%%J-4,HNFC_moving%%J-5,HNFC_moving%%J-6,HNFC_moving%%J-7,HNFC_moving%%J-8,HNFC_moving%%J-9,HNFC_moving%%J-10,HNFC_moving%%J-11,HNFC_moving%%J-12,HNFC_moving%%J-13,nb_vers_hospit%%J-1,nb_vers_hospit%%J-2,nb_vers_hospit%%J-3,nb_vers_hospit%%J-4,nb_vers_hospit%%J-5,nb_vers_hospit%%J-6,nb_vers_hospit%%J-7,nb_vers_hospit%%J-8,nb_vers_hospit%%J-9,nb_vers_hospit%%J-10,nb_vers_hospit%%J-11,nb_vers_hospit%%J-12,nb_vers_hospit%%J-13,nb_vers_hospit%%mean_7J,nb_vers_hospit%%std_7J,nb_vers_hospit%%mean_14J,nb_vers_hospit%%std_14J,O3_FR26005,O3_FR26010,PM10_FR26005,PM10_FR26010,PM10_FR26014,PM10_FR26094,PM25_FR26005,PM25_FR26094,NO2_FR26005,NO2_FR26010,NO2_FR26014,NO2_FR26094,O3_FR26005%%J-1,O3_FR26005%%J-2,O3_FR26005%%J-3,O3_FR26005%%J-4,O3_FR26005%%J-5,O3_FR26005%%J-6,O3_FR26005%%J-7,O3_FR26005%%J-8,O3_FR26005%%J-9,O3_FR26005%%J-10,O3_FR26005%%J-11,O3_FR26005%%J-12,O3_FR26005%%J-13,O3_FR26005%%mean_7J,O3_FR26005%%std_7J,O3_FR26005%%mean_14J,O3_FR26005%%std_14J,O3_FR26010%%J-1,O3_FR26010%%J-2,O3_FR26010%%J-3,O3_FR26010%%J-4,O3_FR26010%%J-5,O3_FR26010%%J-6,O3_FR26010%%J-7,O3_FR26010%%J-8,O3_FR26010%%J-9,O3_FR26010%%J-10,O3_FR26010%%J-11,O3_FR26010%%J-12,O3_FR26010%%J-13,O3_FR26010%%mean_7J,O3_FR26010%%std_7J,O3_FR26010%%mean_14J,O3_FR26010%%std_14J,PM10_FR26005%%J-1,PM10_FR26005%%J-2,PM10_FR26005%%J-3,PM10_FR26005%%J-4,PM10_FR26005%%J-5,PM10_FR26005%%J-6,PM10_FR26005%%J-7,PM10_FR26005%%J-8,PM10_FR26005%%J-9,PM10_FR26005%%J-10,PM10_FR26005%%J-11,PM10_FR26005%%J-12,PM10_FR26005%%J-13,PM10_FR26005%%mean_7J,PM10_FR26005%%std_7J,PM10_FR26005%%mean_14J,PM10_FR26005%%std_14J,PM10_FR26010%%J-1,PM10_FR26010%%J-2,PM10_FR26010%%J-3,PM10_FR26010%%J-4,PM10_FR26010%%J-5,PM10_FR26010%%J-6,PM10_FR26010%%J-7,PM10_FR26010%%J-8,PM10_FR26010%%J-9,PM10_FR26010%%J-10,PM10_FR26010%%J-11,PM10_FR26010%%J-12,PM10_FR26010%%J-13,PM10_FR26010%%mean_7J,PM10_FR26010%%std_7J,PM10_FR26010%%mean_14J,PM10_FR26010%%std_14J,PM10_FR26014%%J-1,PM10_FR26014%%J-2,PM10_FR26014%%J-3,PM10_FR26014%%J-4,PM10_FR26014%%J-5,PM10_FR26014%%J-6,PM10_FR26014%%J-7,PM10_FR26014%%J-8,PM10_FR26014%%J-9,PM10_FR26014%%J-10,PM10_FR26014%%J-11,PM10_FR26014%%J-12,PM10_FR26014%%J-13,PM10_FR26014%%mean_7J,PM10_FR26014%%std_7J,PM10_FR26014%%mean_14J,PM10_FR26014%%std_14J,PM10_FR26094%%J-1,PM10_FR26094%%J-2,PM10_FR26094%%J-3,PM10_FR26094%%J-4,PM10_FR26094%%J-5,PM10_FR26094%%J-6,PM10_FR26094%%J-7,PM10_FR26094%%J-8,PM10_FR26094%%J-9,PM10_FR26094%%J-10,PM10_FR26094%%J-11,PM10_FR26094%%J-12,PM10_FR26094%%J-13,PM10_FR26094%%mean_7J,PM10_FR26094%%std_7J,PM10_FR26094%%mean_14J,PM10_FR26094%%std_14J,PM25_FR26005%%J-1,PM25_FR26005%%J-2,PM25_FR26005%%J-3,PM25_FR26005%%J-4,PM25_FR26005%%J-5,PM25_FR26005%%J-6,PM25_FR26005%%J-7,PM25_FR26005%%J-8,PM25_FR26005%%J-9,PM25_FR26005%%J-10,PM25_FR26005%%J-11,PM25_FR26005%%J-12,PM25_FR26005%%J-13,PM25_FR26005%%mean_7J,PM25_FR26005%%std_7J,PM25_FR26005%%mean_14J,PM25_FR26005%%std_14J,PM25_FR26094%%J-1,PM25_FR26094%%J-2,PM25_FR26094%%J-3,PM25_FR26094%%J-4,PM25_FR26094%%J-5,PM25_FR26094%%J-6,PM25_FR26094%%J-7,PM25_FR26094%%J-8,PM25_FR26094%%J-9,PM25_FR26094%%J-10,PM25_FR26094%%J-11,PM25_FR26094%%J-12,PM25_FR26094%%J-13,PM25_FR26094%%mean_7J,PM25_FR26094%%std_7J,PM25_FR26094%%mean_14J,PM25_FR26094%%std_14J,NO2_FR26005%%J-1,NO2_FR26005%%J-2,NO2_FR26005%%J-3,NO2_FR26005%%J-4,NO2_FR26005%%J-5,NO2_FR26005%%J-6,NO2_FR26005%%J-7,NO2_FR26005%%J-8,NO2_FR26005%%J-9,NO2_FR26005%%J-10,NO2_FR26005%%J-11,NO2_FR26005%%J-12,NO2_FR26005%%J-13,NO2_FR26005%%mean_7J,NO2_FR26005%%std_7J,NO2_FR26005%%mean_14J,NO2_FR26005%%std_1

In [14]:
# baseTabularDataset.enc_X_train.to_csv('enc_X_train.csv', index=False)
# baseTabularDataset.enc_X_test.to_csv('enc_X_test.csv', index=False)
# baseTabularDataset.enc_X_val.to_csv('enc_X_val.csv', index=False)
# baseTabularDataset.fetch_data(save=False)
# dataset = baseTabularDataset.get_dataset(from_date=dt.datetime.strptime('15-01-2019', '%d-%m-%Y'), to_date=dt.datetime.strptime('30-12-2023', '%d-%m-%Y'), shift=[1, 2, 3, 4, 5, 6, 7], rolling_window=[7, 14], create_X_y=True, split_config = {'test_size': 0.2, 'val_size': 0.2, 'shuffle': False}, encoding_pipeline=pipeline, freq='1D', targets_names=target_colomns)

print(dataset.enc_X_train.columns.to_list())
print(dataset.y_train.columns.to_list())
dataset.save_dataset()
dataset2 = dataset.get_dataset(features_names=['O3_FR26005%%J-1', 'confinement1%%J-5##Total_CHU_Dijon'])
print(dataset2.data.columns.to_list())
print(dataset2.train_set.columns.to_list())
print(dataset2.X_train.columns.to_list())
print(dataset2.y_train.columns.to_list())
print(dataset2.enc_X_train.columns.to_list())

dataset.plot(max_subplots=16)
print(dataset.data)
dataset.save_data(root_dir, "data")

baseTabularDataset.plot(from_date='01-01-2022', to_date='15-01-2022', freq='1D', max_subplots=4)
dataset.encode(pipeline=piepline)

dataset.save_data(root_dir, "data")

print(baseTabularDataset.data.info())
baseTabularDataset.encode(encoders=encoders)

print(baseTabularDataset.encoded_data)
print(baseTabularDataset.features)